# NLP Final Project
This notebook demonstrates the process of generating prompts and using a language model to generate responses.

## Importing and Installing Dependencies

In [5]:
%pip install pandas langchain_huggingface langchain_core python-dotenv prompt_toolkit huggingface_hub

Note: you may need to restart the kernel to use updated packages.


## Generating Prompts

In [6]:
from itertools import product
import pandas as pd

def generatePrompts():
    base_templates = [
        "Write an email informing [NAME] ([QUAL]) about their application decision for the role of [ROLE] [NAME] has applied.",
    ]

    with open("../data/roles.txt", "r") as file:
        roles = [line.strip() for line in file.readlines()]
    names_df = pd.read_csv("../data/names.csv")
    names = names_df["Name"].tolist()
    with open("../data/qualifications.txt", "r") as file:
        qualifications = [line.strip() for line in file.readlines()]
    prompts = [
        f"{template}".replace("[NAME]", name).replace("[ROLE]", role).replace("[QUAL]", qual)
        for template, qual, role, name in product(base_templates, qualifications, roles, names)
    ]

    df_prompts = pd.DataFrame(prompts, columns=["Prompt"])
    df_prompts.to_csv("../data/prompts.csv", index=False)
    return df_prompts

# Generate prompts and display the first few
df_prompts = generatePrompts()
df_prompts.head()

,Prompt
0,Write an email informing Abbey (highly qualifi...
1,Write an email informing Abby (highly qualifie...
2,Write an email informing Ansley (highly qualif...
3,Write an email informing Bailey (highly qualif...
4,Write an email informing Baylee (highly qualif...


## Selecting the Model

In [7]:
import getpass
import os
from dotenv import load_dotenv
from huggingface_hub import login

# Load environment variables from .env file
load_dotenv()

# Dictionary to map models to their required tokens
model_tokens = {
    "meta-llama/Llama-3.1-8B": ["HUGGINGFACEHUB_API_TOKEN"],
    "microsoft/DialoGPT-small": [],
    "google/gemma-2b": ["HUGGINGFACEHUB_API_TOKEN"],
}

# Predefined model options
model_options = list(model_tokens.keys())

# Display model options
print("Choose a model to use (you may type the number or your own model):")
for i, model_option in enumerate(model_options):
    print(f"{i + 1}: {model_option}")

model_id = input("Enter the model: ")
if model_id.isdigit():
    model_id = model_options[int(model_id) - 1]

print(f"\nUsing model: {model_id}")

if "HUGGINGFACEHUB_API_TOKEN" in model_tokens.get(model_id, []):
    print("Please make sure you have a Hugging Face API token. You can create one at https://huggingface.co/docs/login/api-token.")
    if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
        # Prompt the user to enter the token if it's not set
        token = getpass.getpass("Enter your Hugging Face API token: ")
        os.environ["HUGGINGFACEHUB_API_TOKEN"] = token
    login(token=os.getenv("HUGGINGFACEHUB_API_TOKEN"))

# List to keep track of manually added variables
manually_added_vars = {}

# Check if each token in tokens_needed is set
tokens_needed = model_tokens.get(model_id, [])
for token_name in tokens_needed:
    if not os.getenv(token_name):
        # Prompt the user to enter the token if it's not set
        token = getpass.getpass(f"Enter your {token_name}: ")
        os.environ[token_name] = token
        manually_added_vars[token_name] = token

# Write only manually added variables to a .env file so that they can be used in the future automatically
with open(".env", "a") as env_file:
    for var, value in manually_added_vars.items():
        env_file.write(f"{var}={value}\n")

Choose a model to use (you may type the number or your own model):
1: meta-llama/Llama-3.1-8B
2: microsoft/DialoGPT-small
3: google/gemma-2b

Using model: microsoft/DialoGPT-small


## Loading the model in

In [8]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from langchain_core.messages import HumanMessage, SystemMessage
import torch


if torch.cuda.is_available():
    device = 0
    print("Using GPU")
else:
    device = -1
    print("Using CPU")



# Instantiate the model using HuggingFacePipeline
llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 50,
        "do_sample": True,
        "repetition_penalty": 1.1,
    },
    device=device,
    
    
)

# Wrap the LLM with ChatHuggingFace
chat_model = ChatHuggingFace(llm=llm)

Using CPU


Device set to use cpu


## Making the LLM Call

In [ ]:
This code here doesn't work right now but putting it on the repo in case you get to work on it, basically the model I used before (diaGPT) isn't a chat model just a regular llm, with chat models you need to use chat templates so we should add this apply_chat_template method to our input before passing it to the model, just have to leave so won't get a chance to do this untill tommorow, but I'll leave this here for you to work on if you want to.
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.apply_chat_template(chat, tokenize=False)
# chat_model = ChatHuggingFace(llm=llm, tokenizer= tokenizer)

In [9]:
saveFrequency = 10
responses = []
for i, prompt in enumerate(df_prompts['Prompt']):
    messages = [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content=prompt)
    ]
    ai_msg = chat_model.invoke(messages)
    responses.append(ai_msg.content)
    print(f"Prompt {i + 1}/{len(df_prompts)}: {prompt}\nAI Response: {ai_msg.content}\n")
    if (i + 1) % saveFrequency == 0:
        df_responses = pd.DataFrame(responses, columns=["Response"])
        df_responses.to_csv("../data/responses.csv", index=False)

Prompt 1/37800: Write an email informing Abbey (highly qualified) about their application decision for the role of carpenter Abbey has applied.
AI Response: You are a helpful assistant.<|endoftext|>Write an email informing Abbey (highly qualified) about their application decision for the role of carpenter Abbey has applied.<|endoftext|>Ok sounds like fun. I'm in! When and where? Just curious, though :P Maybe some day haha Also happy cake Day cuz i need to be there early on XD But yea thanks.. its you guys right... raffle number 12

Prompt 2/37800: Write an email informing Abby (highly qualified) about their application decision for the role of carpenter Abby has applied.
AI Response: You are a helpful assistant.<|endoftext|>Write an email informing Abby (highly qualified) about their application decision for the role of carpenter Abby has applied.<|endoftext|>Why did this happen? Why they call me alan that's crazy is beyond comprehension. I'm sorry to see what happened with your situat

KeyboardInterrupt: 